<a href="https://colab.research.google.com/github/ebamberg/TechPackersColab/blob/main/examples_llama3_1_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --upgrade
!pip install bitsandbytes
!pip install accelerate
!pip install torch

Llama3-8bn needs 16GB of GPU memory.FreeColab Notebook only provides 15GB GPU memory at that moment. So we need to quantize that. See model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True}

In [2]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

First of all a very simple prompt

In [5]:
output= pipeline ("What is the capitol of France?")
print (output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'What is the capitol of France? Paris\nWhat is the capital of Italy? Rome\n'}]


Now use a more complete example on how to build a prompt.
See https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1 for details about the prompt format.

In [11]:

messages = [
    {"role": "system", "content": "Answer questions"},
    {"role": "user", "content": "What is 1+1? Please answer with just the result."},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print (outputs)
print(outputs[0]["generated_text"][len(prompt):],end='.')

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nAnswer questions<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is 1+1? Please answer with just the result.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n2'}]
2.